# Evaluación de modelo: k-Fold Cross Validation

## k-Fold Cross Validation

Sabemos que las observaciones con las que contamos son sólo una parte (una muestra) de todas las que podrían existir en la realidad; por lo tanto  para tener una idea sobre qué performance alcanzará nuestro modelo al generalizar en los casos que no conocemos sabemos que debemos evaluarlo frente  observaciones que no hayan formado parte del Train Set, es por eso que dividimos el Dataset original en Train y Test; entrenamos en el Train Set y evaluamos en el Test Set.

En Scikit-learn hemos utilizado la función **train_test_split()** que forma parte de la librería **sklearn.model_selection** para hacerlo.

Si bien la división entre Train Set y Test Set se hace al azar, siempre puede quedarnos la duda sobre si *de casualidad* el Train Set o el Test Set no habrán tenido alguna característica especial que hiciera que al evaluar, el resultado obtenido no sea representativo de las observaciones que no tenemos.  

También podemos pensar qué pasaría si hubiéramos efectuado la división entre Train y Test eligiendo otras observaciones en cada una de ellas (por ejemplo usando otra semilla). La métricas de evaluación, por ejemplo Accuracy si era un problema de clasificación o RMSE si era de regresión,  hubieran dado igual? Idénticas no van a dar ...entonces cuál medida es mejor, si en ambos casos el train y el test se eligieron al azar? 
Obviamente que ninguna puede ser mejor ... pero es demostrable que:  

> **el promedio de ambas sí será una mejor aproximación a la realidad!**   

Entonces lo que podemos hacer es efectuar varias selecciones distintas de Train y Tests sets, en cada caso calcular la métrica de evaluación que deseemos y luego promediarlas y ese valor será una mejor aproximación a lo que ocurrirá en las observaciones desconocidas que no forman parte de nuestros datos.  



Hay varias maneras de efectuar este proceso: 

- podemos hacerlo "a mano", por ejemplo cambiando la semilla del train_test_split dentro de un for

- podemos usar **k Fold Cross Validation** (se suele decir simplmente Cross Validation o CV), que será el método que veremos ahora y que forma parte de sklearn y es el más usado

- podemos usar **Bootstraping** que es muy similar a CV, pero que no viene dentro del paquete de sklearn.  

> Observar que k-fold Cross Validation o las otras opciones mencionadas **no son una forma de mejorar el rendimiento de nuestro modelo**, sino que nos permiten tener **más confianza en el resultado de la métrica usada para evaluarlo**.

> Tener en cuenta también que cuantas más veces repitamos la opearación de elegir nuevos train y test sets para luego evaluar obviamente se va a **multiplicar en la misma medida** el tiempo de cómputo, lo cual puede hacer que sea inaplicable para datasets muy grandes o algoritmos complejos;    

> pero también es cierto que cuantas más veces repitamos la operación y promediemos los resultados, **mayor confianza** tendremos en el resultado obtenido! 

En el nombre, k Fold Cross Validation, la k indica la cantidad de veces que se repetirá el proceso, cada repetición se denomina fold:

> Valores típicos para k son 3, 5 y 10.



Técnicamente se dice que hay una **varianza grande**, cuando el resultado obtenido depende en gran medida de las observaciones que por azar caen en el Train Set y Test Set.

### Cross Validation para evaluar un modelo (sin selección de modelo o de hiperparámetros)

Supongamos que ya sabemos qué modelo queremos usar y también sabemos qué valor de hiperparámetros usaremos, es decir que nuestro interés es evaluar al modelo con alguna métrica.

Podría ser que en el DataSet original hubiera algunas observaciones "más difíciles" o "más fáciles" de pronosticar y desearíamos que todas ellas estuvieran en el Test Set, es por esto que al implementar k-fold cv se tiene la precaución de que cada observación forme parte de algún Test Set, como se muestra en la siguiente figura para 5-fold cv.  

Si elegimos un valor de k=5 entonces en k fold Cross Validation se dividirá el dataset en 5 partes iguales, cada "fold" se conformará de la siguiente manera: cada parte hará un vez de Test Set y las restantes hará de Train Set, como se muestra en la siguiente figura. Observe que **cada observación estará una vez dentro de un Test Set**, lo cual es bueno que ocurra.

![k-fold Cross Validation](https://i.ibb.co/7jqMWKF/k-fold-Cross-Validation.png)

En cada fold se entrena y se calcula la métrica deseada (por ej: para Regresión RMSE o $R^2$, o para Clasificación Accuracy o F1), luego se promedian las 5 métricas obtenidas.  

> El promedio de las evaluaciones será una mejor evaluación que cualquiera de las individuales y por lo tanto tendremos un resultado más cercano al que ocurrirá al enfrentar nuestro modelo frente a las observaciones que no conocemos.

#### Qué DataSet utilizar?

Como en este caso ya sabemos qué modelo queremos usar y tenemos definidos sus hiperparámetros sólo queremos evaluar al modelo. En este caso debemos pasar como dato todo el DataSet ya que será el proceso de k fold CV el que efecutará las necesarias divisiones entre Train y Test.

### Pero qué parámetros del modelo usar?

Muchas veces suelen generarse dudas como la siguiente:

Supongamos que el modelo elegido es una Regresión Lineal (estamos pensando una situación sin selección de modelo ni de hiperparámetos, ya hemos elegido el modelo)


$$h(x_1,x_2 ...,x_n)=w_0  + w_1 x_1 + w_2 x_2 + ... + w_n x_n $$

 y deseamos saber qué error RMSE se cometerá al generalizar:  

para cada uno de los 5 folds corridos se entrenaría en un Train Set distinto por lo cual en cada uno de estos casos se obtendría un conjunto de valores **distintos** para los parámetros $w_i$, luego de correr las 5 veces tendríamos en definitiva 5 soluciones posibles para los coeficientes $w_i$: entonces, una duda muy habitual que se plantea es qué parámetros utilizar? Acaso los que corresponden al fold con menor RMSE?

La respuesta es **ninguno** de los conjuntos de parámetros obtenidos para cada fold.  

> k-fold cross validation nos sirve para estimar cuál será el error que obtendrá el modelo al generalizar con la mayor confianza posible. Una vez que calculamos el promedio de los RMSE **procederemos a entrenar el modelo con todos los datos del DataSet original** y ésos serán los parámetros $w_i$ de nuestro modelo, **de la misma manera que hacíamos en la materia Introducción a la IA** cuando sólo usábamos un Train Set y un Test Set. 


### Los Casos Extremos

- k-fold Cross Validation con un valor de **k=1** no es otra cosa más que la simple división en Train Set y Test Set que usamos en Introducción a la IA, ahora sabemos que tendría una gran varianza ya que los resultados del único error calculado podrían variar mucho según la elección del único Train y Test set, generalmente se le denomina simplemente como Cross Validation.  


- k-fold Cross Validation con un valor de **k=m siendo m la cantidad de observaciones** del DataSet original es el otro extremo, en este caso en cada uno de los folds  cada Train tendrá m-1 observaciones y cada Test sólo 1 observación para pronosticar! A este caso se lo denomina **Leave one out Cross Validation Error (LOO-xVE)**. Es la situación con **menor varianza** posible para la evaluación del error para un dataset dado, pero obviamente la que demandará más tiempo. Existen algoritmos para disminuir el tiempo de cómputo como el de Vizier.   

**Finalmente**  


> Si es posible, usar Leave one out cv, sino usar k-fold cv con el mayor valor de k posible, sino usar cv simple con un solo Train y Test set.


###  cross_val_score : uso de CV para evaluar un modelo con sklearn:

Scikit-learn nos ofrece varias herramientas relacionadas con cv, la más simple de ellas efectúa por sí sola las conformaciones de cada uno de los folds, entrena el modelo en cada uno de ellos, nos permite elegir la métrica de evaluación deseada y nos brinda los resultados de la misma para cada uno de los folds.  Luego nosotros tenemos que calcular el promedio de las evaluaciones.  

La documentación oficial se encuentra aquí: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html  

La sintaxis es la siguiente:  

~~~
sklearn.model_selection.cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
~~~

Donde las principales opciones son las siguientes:

- **estimator**: es el modelo que hemos creado anteriormente  
- **X e y** son el Dataset sobre el que queremos evaluar el modelo.  
- **cv**: la cantidad de folds que queremos que utilice  
- **scoring**: la métrica de evaluación que deseamos utilizar, por defecto el valor es None lo cual significa que utilizará el score por defecto que tenga el modelo, que para el caso de Clasificación suele ser AC y para el de Regresión suele ser $R^2$. Pero si lo deseamos podemos indicarle otro, pero **sólo podemos usar uno**.

En la siguiente página oficial https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter podemos ver los principales scores que se pueden calcular, los más usados:  

Para Clasificación:  

- ‘accuracy’
- ‘f1’
- ‘f1_micro’
- ‘f1_macro’
- ‘precision’ 
- ‘recall’

Para Regresión:  

- ‘r2’
- ‘neg_mean_squared_error’ (MSE con signo -, uno después tiene que multiplicar por -1)
- ‘neg_root_mean_squared_error’ (RMSE con signo - , uno después tiene que multiplicar por -1)







Veamos cómo utilizarlo.

En el archivo data/datos1.csv tenemos un dataset con datos ficticios, pero que servirán a nuestros fines. Tenemos 3 columnas con features o variables explicativas, x1, x2 y x3. Y una columna y que es la que queremos pronosticar con dos clases o labels que hemos puesto como valores 0 y 1. En total son 50 instancias u observaciones.

In [ ]:
import pandas as pd
df=pd.read_csv('data/datos1.csv')
df.head()

,x1,x2,x3,y
0,-0.674774,-1.368211,-0.174910,0
1,2.721351,-3.060436,-3.259069,1
2,-2.270269,-0.196512,1.448112,0
3,-0.989482,-0.553864,0.414766,0
4,1.043243,0.690214,-0.388195,1


Separamos en X e y como nos suele pedir sklearn:

In [ ]:
X=df.drop('y', axis='columns')
y=df['y']


Supongamos que ya hemos decidido que nuestro modelo será Árbol de Decisión con hiperparámetro de profundidad 4 y deseamos medir la Exactitudo o Accuracy del modelo.

Creamos el modelo de árbol:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
arbol=DecisionTreeClassifier(max_depth=4)

Ahora efectuamos el proceso de k fold Cross Validation.   
Supongamos que queremos utilizar 5 folds.  

**No necesitamos dividir el DataSet (X,y) en train y test**, porque éso lo hará automáticamente cv, de hecho lo hará 5 veces, una por cada fold!

In [ ]:
from sklearn.model_selection import cross_val_score 

In [ ]:
AC_scores = cross_val_score(arbol, X, y, cv=5)

# veamos los resultados
print("AC en cada fold: ",AC_scores)

AC en cada fold:  [0.9 0.9 1.  1.  0.9]


Observemos todo lo que hizo cross_val_score: 

- ha creado los 5 folds
- en cada uno de ellos hizo la división Train Test (80% y 20% de las observaciones respectivamente porque le pedimos 5 folds)
- en cada uno de ellos entrenó al modelo ( efectuó .fit en el train que le correspondía)
- calculó la Accuracy en cada fold  

Bastantes tareas juntas!

#### Evaluación del modelo con cv:

Como ahora conocemos AC para cada uno de los folds, si promediamos sus valores tendremos un valor de AC más confiable que el que obteníamos antes:

In [ ]:
AC_media=AC_scores.mean()
AC_media

0.9400000000000001

pero además, podemos calcular el Desvío Standard que hay entre los valores de AC calculados.   

Recordemos que si asumimos una distribución normal o gaussiana para los resultados de los AC:   

- en el intervalo media +/- un desvío standard deberíamos tener el 68,2% de todas las mediciones de AC
- en el intervalo media +/- 2 desvíos standard deberíamos tener el 95,4% de todas las mediciones de AC 
- en el intervalo media +/- 3 desvíos standard deberíamos tener el 99,6% de todas las mediciones de AC

Así que generalmente se brinda la media +- alguna de las 3 posibilidades anteriores, indicando a qué porcentaje corresponde si es necesario: 


In [ ]:
desvio=AC_scores.std()
desvio

0.04898979485566355

In [ ]:
print("AC= ",AC_media, " +/- ",desvio, " (68%)" )
print("AC= ",AC_media, " +/- ",2*desvio, " (95%)" )
print("AC= ",AC_media, " +/- ",3*desvio, " (99%)" )

AC=  0.9400000000000001  +/-  0.04898979485566355  (68%)
AC=  0.9400000000000001  +/-  0.0979795897113271  (95%)
AC=  0.9400000000000001  +/-  0.14696938456699066  (99%)


- Esto significa que entre (0.94 - 0.1469) y (0.94 + 0.1469) tenemos un 99% de probabilidades de que se encuentre el verdadero valor de AC!  Esta información no la podíamos brindar cuando sólo hacíamos una división entre Train y test.   

- Observe que ésto no mejora la performance del modelo, sino que nos permite tener una mayor confianza en el resultado obtenido.  

- En lineas generales cuántos más folds usemos mejor, pero debemos tener en cuenta que así también se multiplicará el tiempo de procesamiento.

Como comentábamos anteriormente, podemos elegir otras métricas para el score, en este caso como la variable a pronosticar asume sólo dos valores (es bivariada), podemos usar f1 ( si fuera un problema multivariado habría que elegir entre f1_micro o f1_macro).   

Calculemos f1, recordemos que el modelo ya lo habíamos instanciado y se llamaba arbol:

In [ ]:
f1_scores = cross_val_score(arbol, X, y, cv=5,scoring='f1' )
f1_scores

array([0.90909091, 0.90909091, 1.        , 1.        , 0.88888889])

In [ ]:
f1_media=f1_scores.mean()
f1_desvio=f1_scores.std()

print("F1= ",f1_media, " +/- ",f1_desvio, " (68%)" )
print("F1= ",f1_media, " +/- ",2*f1_desvio, " (95%)" )
print("F1= ",f1_media, " +/- ",3*f1_desvio, " (99%)" )

F1=  0.9195959595959596  +/-  0.07521448431757036  (68%)
F1=  0.9195959595959596  +/-  0.15042896863514071  (95%)
F1=  0.9195959595959596  +/-  0.22564345295271107  (99%)


Lo único malo que tiene cross_val_score es que si queremos calcular varias métricas para evaluar, debemos hacerlo de a una por vez, lo cual puede demorar mucho tiempo ya que para cada un sería un nuevo procesamiento de todo el modelo!

### cross_validate: otra forma de evaluar el modelo:  

cross_validate sirve como el anterior, para efectuar Cross Validation, pero es más potente, nos brinda más información en el resultado y también nos permite calcular varios scores:

Documentación oficial: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate

Sintaxis:
~~~
sklearn.model_selection.cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, error_score=nan)
~~~

Las opciones principales son: 

- **estimator**: es el modelo que hemos creado anteriormente  
- **X e y** son el Dataset sobre el que queremos evaluar el modelo.  
- **scoring**= idem anterior, pero se puede pasar un score **o varios en un array** 
- **cv**=  cantidad de folds, por defecto 5
- **return_train_score**= False por defecto, Falso, pero puede ser de interés obtener también los valores de score correspondientes al train set en cada fold.
- **return_estimator**: si queremos que devuelve el modelo obtenido en cada fold.


Resultados: 

En esencia devuelve los scors obtenidos en cada fold:

- test_score
- train_score
- fit_time
- score_time
- estimator



In [ ]:
from sklearn.model_selection import cross_validate

Los datos ya los tenemos de antes son X, e y y también el modelo de Árbol, así que podemos ir directamente a cv con cross_validate, hagamos algo sencillo:


In [ ]:
scores_2 = cross_validate(arbol, X, y, cv=5)

In [ ]:
scores_2

{'fit_time': array([0.0030098 , 0.00199485, 0.00199294, 0.00197792, 0.00200176]),
 'score_time': array([0.00097919, 0.        , 0.        , 0.00102854, 0.0009737 ]),
 'test_score': array([0.9, 0.8, 1. , 1. , 0.9])}

Observe que la estructura devuelta es la de un diccionario.

Ahora pasémosle un par de métricas:

In [ ]:
scores_2 = cross_validate(arbol, X, y, cv=5, scoring=['accuracy','f1','precision', 'recall'])
scores_2

{'fit_time': array([0.00298977, 0.00301671, 0.00199842, 0.00299001, 0.00398874]),
 'score_time': array([0.00598574, 0.00695491, 0.00398803, 0.00398993, 0.00598621]),
 'test_accuracy': array([0.9, 0.8, 1. , 1. , 0.9]),
 'test_f1': array([0.90909091, 0.8       , 1.        , 1.        , 0.88888889]),
 'test_precision': array([0.83333333, 0.8       , 1.        , 1.        , 1.        ]),
 'test_recall': array([1. , 0.8, 1. , 1. , 0.8])}

Cuando necesitemos acceder a sólo uno de los ítems, lo podemos hacer como en cualquier diccionario.  Por ejemplo si queremos acceder a los valores de Accuracy

In [ ]:
scores_2['test_accuracy']

array([0.9, 0.8, 1. , 1. , 0.9])

O si queremos acceder a los valores de f1 obtenidos en cada fold:

In [ ]:
scores_2['test_f1']

array([0.90909091, 0.8       , 1.        , 1.        , 0.88888889])

Obtengamos igual que antes el promedio y el desvío standard para Accuracy y F1:

In [ ]:
AC_media2=scores_2['test_accuracy'].mean()
AC_desvio2=scores_2['test_accuracy'].std()

f1_media2=scores_2['test_f1'].mean()
f1_desvio2=scores_2['test_f1'].std()


print("AC2= ",AC_media2, " +/- ",AC_desvio2, " (68%)" )
print("AC2= ",AC_media2, " +/- ",2*AC_desvio2, " (95%)" )
print("AC2= ",AC_media2, " +/- ",3*AC_desvio2, " (99%)" )
print('--------------------------------------------------')
print("F12= ",f1_media2, " +/- ",f1_desvio2, " (68%)" )
print("F12= ",f1_media2, " +/- ",2*f1_desvio2, " (95%)" )
print("F12= ",f1_media2, " +/- ",3*f1_desvio2, " (99%)" )

AC2=  0.9200000000000002  +/-  0.0748331477354788  (68%)
AC2=  0.9200000000000002  +/-  0.1496662954709576  (95%)
AC2=  0.9200000000000002  +/-  0.22449944320643642  (99%)
--------------------------------------------------
F12=  0.9195959595959596  +/-  0.07521448431757036  (68%)
F12=  0.9195959595959596  +/-  0.15042896863514071  (95%)
F12=  0.9195959595959596  +/-  0.22564345295271107  (99%)


#### Evaluando en el Train?

Si bien sabemos que el rendimiento real de nuestro modelo se debe medir sobre el Test set en cada uno de los folds, también puede ser útil ( y lo será más adelante) evaluar en  el Train ... será una ayuda para saber si estamos cometiendo overfiting ...

In [ ]:
scores_3 = cross_validate(arbol, X, y, cv=5, scoring=['accuracy'], return_train_score=True)
scores_3

{'fit_time': array([0.00299096, 0.00199652, 0.00199246, 0.00197935, 0.00201964]),
 'score_time': array([0.0009973 , 0.00199747, 0.00103617, 0.00199437, 0.00097227]),
 'test_accuracy': array([0.9, 0.9, 1. , 1. , 0.9]),
 'train_accuracy': array([1., 1., 1., 1., 1.])}

# Bibliografía

https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation